In [2]:
# import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

In [3]:
class HVC_RA(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, layer_dim):
        super(HVC_RA, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.HVC = nn.RNN(input_size, hidden_size, nonlinearity='relu')
        self.RA = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        output, hidden = self.HVC(x, hidden)
        output = self.RA(output)
        return output, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.hidden_size)
        return hidden
    
    def loss_function():
        '''
        Purpose: 
            - to calculate the difference between the HVC neuron activation, and proper RA output 
            - Uses MSE loss
    
        Input:
            - HVC neuron activation
            - RA output

        Output:
            - MSE loss
        '''

        

        pass

In [26]:
class Training():
    def __init__(self):
        pass
        
    def generate_syllables(self, syllable_count):
        '''
        Purpose: 
            - Generate syllables for training
            - 4 params for syllable generation, freq, amplitude, y-offset, length
        Input: 
            - Number of syllables to generate
        Output: 
            - returns songs as a numpy array of shape (syllable_count * individual lengths, 3)
            - returns length of eachs syllable 
            - returns the HVC input to the network for each syllable
        ''' 
        
        # careful, the code below is cringe af!
        syllable_duration = np.random.uniform(3, 10, syllable_count)
        syllable_duration = np.array(syllable_duration, dtype=int)

        total_syllable_length = np.sum(syllable_duration)

        syllables = np.zeros((total_syllable_length, 3))
        self.syllable_index = np.zeros((total_syllable_length, 1))
        
        cum_length = 0
        for i, length in enumerate(syllable_duration):
            self.syllable_index[cum_length:cum_length+length] = i
            freq = np.random.uniform(0, 1)
            amp = np.random.uniform(0, 1)
            y_offset = np.random.uniform(0, 1)
            for t in range(length):
                syllables[cum_length, 0] = freq
                syllables[cum_length, 1] = amp
                syllables[cum_length, 2] = y_offset
                cum_length += 1

        self.syllables = syllables

        HVC_on = np.ones((total_syllable_length, 1))
        HVC_off = np.zeros((total_syllable_length, 1))

        self.HVC_on = HVC_on
        self.HVC_off = HVC_off
        self.num_syllables = syllable_count
        # isomorphic to the syllables, but is a 1d numpy list of the syllable index (same in length as syllables)
        

        return syllables, syllable_duration, HVC_on, HVC_off

    def generate_train_set(self, train_size):
        '''
        Purpose:
            - Generate a training set for the model
        Input:
            - train_size: number of training examples to generate
        Output:
            - train_set: a list of training examples
        '''
        # create a list of syllables the length of self.num_syllables, each syllables is a random length between 1 syllable and all of them
        train_set = []

        for i in range(train_size):
            padded_syllables = np.zeros((self.syllables.shape[0], 3))
            padded_HVC_on = np.zeros((self.syllables.shape[0], 1))


            ## vERITFY THIS IS CORRECT

            length = np.random.randint(1, self.num_syllables)
            # get index where the int after length starts (i.e. if length = 2, then index = 3) in the syllableIndex
            print(length)
            index = np.where(self.syllable_index == (length))[0][0]
            
            # order has to remain the same
            padded_HVC_on[:index] = self.HVC_on[:index]
            padded_syllables[:index] = self.syllables[:index]

            ### END VERIFICATION


    def train(self, model, epochs, train_size):
        '''
        Purpose:
            - Train the model
        Input:
            - model
            - train_loader
            - epochs
        Output:
            - trained model
        ''' 
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        self.generate_train_set(train_size)



        # for epoch in range(epochs):


        pass

    def get_syllables(self):
        return self.syllables

In [27]:
train = Training()

syllables, syllable_duration, HVC_on, HVC_off= train.generate_syllables(4)

syllables = train.get_syllables()

model = HVC_RA(3, 10, 1, 1)


train.train(model, epochs=10, train_size=1000)

3
2
1
1
3
1
3
3
1
2
2
1
2
3
1
3
1
3
1
2
1
3
2
2
1
2
2
3
2
3
1
2
2
1
3
1
1
3
2
3
2
3
3
2
3
1
2
2
3
2
3
2
3
3
2
3
3
2
2
1
1
2
2
2
3
3
2
1
2
3
2
2
1
2
2
3
1
1
3
2
1
1
1
2
3
3
2
3
1
2
3
2
2
2
3
3
3
3
2
3
1
2
1
2
2
1
2
1
3
3
2
1
1
2
1
2
3
3
3
2
1
2
3
2
3
3
1
1
1
2
2
3
1
2
1
1
2
2
3
2
2
3
2
1
2
1
3
3
2
1
2
2
3
1
2
2
1
1
3
2
1
1
1
3
3
2
3
1
2
1
3
1
2
3
3
3
1
3
1
2
1
3
1
1
3
2
3
3
1
1
1
1
2
3
2
1
3
3
2
1
3
1
2
2
2
1
2
3
2
2
3
3
1
2
3
2
3
3
3
1
3
3
1
1
3
2
3
2
3
3
3
3
3
3
1
3
2
3
3
2
3
3
2
2
3
2
3
1
3
1
1
2
2
3
1
3
3
2
3
2
3
1
1
2
2
3
2
1
1
3
2
3
2
2
2
1
2
2
2
1
1
2
3
3
3
1
1
1
3
2
1
2
3
3
1
1
3
2
1
1
3
3
1
1
2
1
1
3
2
1
2
2
2
3
1
3
1
3
3
1
3
1
3
3
2
1
3
2
3
2
2
3
1
2
1
3
3
3
2
3
2
2
2
2
3
3
3
1
1
1
3
3
1
3
1
1
3
1
1
1
1
3
2
3
1
3
1
2
1
3
2
2
2
3
1
3
2
1
3
3
3
2
1
2
2
3
3
1
2
1
1
3
3
2
3
1
2
1
2
2
2
3
1
1
2
3
1
1
3
3
3
3
3
2
2
1
1
1
1
3
2
1
1
2
2
2
3
3
3
1
3
1
2
3
1
2
1
3
3
3
3
1
2
1
1
1
1
2
1
3
1
2
2
2
2
2
1
3
2
2
2
2
1
1
3
1
3
3
2
2
2
2
3
3
2
2
2
2
2
3
1
3
3
3
1
1
2
1
1
2
2
2
1
3
1
3
1
2
1
3
